<a href="https://colab.research.google.com/github/akshaygrao77/DeepLearning-Assignment2/blob/main/Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > dataset.zip
!unzip dataset.zip

In [2]:
!rm dataset.zip

**IMPORT LIBRARIES**

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from tensorflow.keras import  models, optimizers, layers, activations
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from keras.callbacks import History
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

# IMPORT PRE_TRAINED MODELS 

In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet import ResNet50
from keras.applications.xception import Xception

In [5]:
!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.7 MB 9.4 MB/s 
     |████████████████████████████████| 181 kB 56.5 MB/s 
     |████████████████████████████████| 144 kB 50.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [6]:
wandb.init(project="DLAssignment2-partb", entity='cs21s002-ee21s113-dlassignment-1')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
#Constants
train_data_path = os.getcwd()+"/inaturalist_12K/train"
test_data_path = os.getcwd()+"/inaturalist_12K/val"
width, height = 200, 200
batch_size = 128
input_shape = (200, 200, 3)

**IMAGE DATA PRE PROCESSING**

In [8]:
def dataset_Generators(train_data_aug=False):
    if train_data_aug:
        train_idg = ImageDataGenerator(
            rescale=1. /255,
            validation_split=0.1,              
            horizontal_flip=True,
            vertical_flip=False,
            rotation_range=15,
            shear_range=0.1,
            zoom_range=0.1,
            width_shift_range=0.2,
            height_shift_range=0.2
        )
    else:
        train_idg = ImageDataGenerator(
            rescale=1. /255,
            validation_split=0.1                 
        )
    
    test_idg = ImageDataGenerator(
            rescale=1. /255
        )
    
    print(f"{train_data_path}")
    train_generator = train_idg.flow_from_directory(
        train_data_path,
        batch_size=batch_size,
        target_size=(width, height),
        subset='training',
        class_mode='categorical'
    )

    val_generator = train_idg.flow_from_directory(
        train_data_path,
        batch_size=batch_size,
        target_size=(width, height),
        subset='validation',
        class_mode='categorical'
    )

    print(f"{test_data_path}")
    test_generator = test_idg.flow_from_directory(
        test_data_path,
        batch_size=batch_size, 
        target_size=(width, height)
    )
    
    return train_generator, val_generator, test_generator

**BUILD A CNN_MODEL ON ALREADY EXISTING PRE TRAINED MODEL**

In [9]:
inception_v3 = "InceptionV3"
inception_resnet_v2 = "InceptionResNetV2"
resnet50 = "ResNet50"
xception = "Xception"

def cnn_Model(model_name=xception, dropout=0.2):
    global base_model

    if model_name == xception:
        base_model = Xception(
            weights="imagenet",  
            input_shape=input_shape,
            include_top=False,
        ) 
    elif model_name == inception_v3:
        base_model = InceptionV3(
            weights="imagenet", 
            input_shape=input_shape,
            include_top=False,
        )  
    elif model_name == inception_resnet_v2:
        base_model = InceptionResNetV2(
            weights="imagenet", 
            input_shape=input_shape,
            include_top=False,
        ) 
    else:
        base_model = ResNet50(
            weights="imagenet", 
            input_shape=input_shape,
            include_top=False,
        )  

    # Freeze the base_model
    base_model.trainable = False

    # Create new model on top
    inputs = keras.Input(shape=input_shape)

    x = base_model(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(dropout)(x)

    model = Sequential()
    model.add(Model(inputs, x))
    model.add(layers.Dense(10))
    model.add(layers.Activation('softmax'))

    return model

In [10]:
sweep_config = {
  'name': 'Assignment 2 - partb',  
  'method': 'grid',
  'parameters': {
        'dataset_aug':{
            'values':[True,False]   
        },
        'dropout':{
            'values':[0.2, 0.5]
        },
        'model_name':{
            'values':[inception_v3, inception_resnet_v2]
        },
        'epochs':{
            'values':[5]
        },
        'lr':{
            'values':[0.0015,0.0001]
        }
    }
}

In [11]:
default_config={
    'dataset_aug':True,
    'dropout':0.2,
    'model_name':inception_v3,
    'epochs':5,
    'lr':0.00001
}

**HYPER PARAMETER FINE TUNING**

In [12]:
def hyperparameter_run():
    #  MirroredStrategy.
    if tf.config.list_physical_devices('GPU'):
        strategy = tf.distribute.MirroredStrategy()
    else: 
        strategy = tf.distribute.get_strategy()
    print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

    wandb.init(config=default_config, magic=True, project="DLAssignment2-partb", entity='cs21s002-ee21s113-dlassignment-1')
    config = wandb.config
    run_name = str(config).replace("{", "").replace("}","").replace(":","-").replace(",","-")
    wandb.run.name = run_name

    # Open a strategy scope and create the model
    with strategy.scope():
        model = cnn_Model(config.model_name, config.dropout)
    
    model.summary()

    plot_model(model)

    model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(), metrics="categorical_accuracy")

    history_callback = History()

    train_generator, val_generator, test_generator = dataset_Generators(True)

    model.fit(train_generator ,epochs=config.epochs, validation_data=val_generator, callbacks=[WandbCallback()])

    base_model.trainable = True

    model.summary()

    model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(config.lr), metrics="accuracy")

    model.fit(train_generator ,epochs=config.epochs, validation_data=test_generator, callbacks=[WandbCallback()])

    model.save(f'{run_name}.h5')  # creates a HDF5 file 'my_model.h5'

In [13]:
sweep_id = wandb.sweep(sweep_config, project="DLAssignment2-partb", entity='cs21s002-ee21s113-dlassignment-1')

Create sweep with ID: eivlxmcl
Sweep URL: https://wandb.ai/cs21s002-ee21s113-dlassignment-1/DLAssignment2-partb/sweeps/eivlxmcl


In [14]:
wandb.agent(sweep_id, function=hyperparameter_run, project="DLAssignment2-partb", entity='cs21s002-ee21s113-dlassignment-1')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: eibuef76 with config:
wandb: 	dataset_aug: True
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	lr: 0.0015
wandb: 	model_name: InceptionV3


Number of devices: 1


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


87924736/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
 activation_94 (Activation)  (None, 10)                0         
                                                                 
Total params: 21,823,274
Trainable params: 20,490
Non-trainable params: 21,802,784
_________________________________________________________________
/content/inaturalist_12K/train
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
/content/inaturalist_12K/val
Found 2000 images belonging to 10 classes.
Epoch 1/5
 3/71 [>.............................] -

wandb: Ctrl + C detected. Stopping sweep.
